# Homework 1 - after Lesson 6 Production Systems

## Question 1: ~2 pages
Rey has managed to capture Snoke and Kylo Ren on the planet Quesh. Quesh has a poisonous atmosphere, however, and so Rey, Snoke, and Kylo Ren will have to be kept together in quarantine for two weeks upon arriving back on the orbiting ship.
Only one shuttle is available to transfer individuals back and forth between Quesh and the orbiting ship, and that shuttle can only seat one person. The shuttle has an autopilot, though, so it can fly without anyone in it.
Leia, in the orbiting ship, refuses to let Rey be alone with Snoke (without Kylo) either on the planet or in quarantine, knowing that Snoke will turn Rey to the dark side. Snoke would rather stay and die than let Rey be alone with Kylo Ren, knowing that she will turn Kylo to the light side. Leia wants Snoke alive, and therefore agrees to his demand that Kylo and Rey never be alone together (without Snoke) either on the planet or in quarantine. It is okay, however, for Kylo & Rey or Rey & Snoke to be together if the shuttle is with them, as long as one of them departs on the shuttle.
In simple terms: the goal is to move Rey, Snoke, and Kylo from the planet to the ship. Only one can move at a time, and the shuttle can move without a passenger. Rey and Kylo can never be alone together without the shuttle, and Rey and Snoke can never be alone together without the shuttle. (If you do are unfamiliar with Star Wars, know that this paragraph contains everything you need to know to solve this problem.)
First, construct a semantic network representing this problem. This should take approximately half a page, including a figure of two states with a transition between them. Make sure to include all components of the state, and an operator indicating how we transition from one state to another.
Second, apply generate & test to this semantic network in order to solve the problem. In applying generate & test, your generator should be smart enough to only make valid moves (e.g. it will not try to move two people at once or make consecutive planet-to-ship moves without the transport ship coming back), but it should not be smart enough to only make moves that result in valid states (e.g. it should still try to move Kylo first, even though that move results in an invalid state). Your tester, in turn, should check each generated state to see if (a) it follows the rules, and (b) if it has met the goal. You may decide whether identifying states that have already been visited is the responsibility of the generator or the tester.
Include the entire semantic network that solves this problem. Clearly indicate which states are failed, and why. The semantic network should explore the entire problem space: every state should be either ruled out or have its following states explored. We expect this will fit on one page: you may create a more space-efficient representation if need be. As long as it is legible, you may also hand-write the network and insert it as an image into your paper.

transfer: Quesh -> orbiting ship with Leia

vehicle: one shuttle - 1 passenger only - autopilot

passengers: Ray, Kylo, Snoke

can be kept together: Rey+Kylo+shuttle, Rey+Snoke+shuttle

can't be kept together: Rey+Snoke w/o Kylo, Rey+Kylo w/o Snoke

Start state: Quesh(Ray, Kylo, Snoke, shuttle), Ship()

End state: Quesh(), Ship(Ray, Kylo, Snoke, shuttle)

In [1]:
@enum Reason begin
            initial   #0
            reachedgoal
            valid
            illegal
            empty
            loop
            repeat
       end

mutable struct Status
  step::Int64
  substep::Int64
  parent::Int64
  passengers::Array{Bool}
  shuttle::Bool
  valid::Bool
  reason::Reason
end

function generate!(nextstep::Int64,nextindex::Int64,statuses::Array{Status}):Int64
    previous::Int64 = nextstep - 1;
    substep = 0
    for j in 1:length(statuses);
        if (statuses[j].step == previous 
            && statuses[j].valid == true )
#           a persons can move if on the same side as the Shuttle
            for k in 1:length(statuses[j].passengers)
                if statuses[j].passengers[k] == statuses[j].shuttle;
                    substep = substep + 1
#                   copy status to a new entry
                    statuses[nextindex+substep].step   = nextstep
                    statuses[nextindex+substep].substep = substep
                    statuses[nextindex+substep].parent = j
#                   person travels with shuttle
                    statuses[nextindex+substep].passengers[k] = ! statuses[j].passengers[k]
#                   persons staying
                    for m in 1:length(statuses[j].passengers)
                        if k != m
                            statuses[nextindex+substep].passengers[m] = statuses[j].passengers[m]
                        end
                    end
#               shuttle departs
                statuses[nextindex+substep].shuttle = ! statuses[j].shuttle
#               default INVALID
                statuses[nextindex+substep].valid  = false
                end
            end
            
#           shuttle can move empty
            substep = substep + 1
#           copy status to a new entry
            statuses[nextindex+substep].step   = nextstep
            statuses[nextindex+substep].substep = substep
            statuses[nextindex+substep].parent  = j
            for m in 1:length(statuses[j].passengers)
               statuses[nextindex+substep].passengers[m] = statuses[j].passengers[m]
            end
            statuses[nextindex+substep].valid   = false
#           shuttle departs (empty)
            statuses[nextindex+substep].shuttle = ! statuses[j].shuttle
        end
    end
    return nextindex+substep
end

function test!(step::Int64,s::Array{Status})
    goalstatusreached::Bool = false
    notanyvalid::Bool = true

    for j in 1:length(s);
        if s[j].step == step;
            s[j].valid = true
#           Rey and Kylo together AND shuttle is not there AND Snoke is not there
            if ( s[j].passengers[1] == s[j].passengers[2]  
              && s[j].passengers[1] != s[j].shuttle 
              && s[j].passengers[1] != s[j].passengers[3] ) ||
#           Rey and Snoke together AND shuttle is not there AND Kylo is not there
               ( s[j].passengers[1] == s[j].passengers[3] 
              && s[j].passengers[1] != s[j].shuttle 
              && s[j].passengers[1] != s[j].passengers[2]  )
                s[j].valid = false
                s[j].reason = illegal
            elseif ( s[j].passengers[1] == s[j].passengers[2] == s[j].passengers[3] ) && s[j].passengers[1] != s[j].shuttle
                s[j].valid = false;
                s[j].reason = empty
            end
#           check if status appears in ancestor steps
            if s[j].valid
                parentindex = s[j].parent
                while parentindex > 0
                    if comparestatuses(s[j],s[parentindex])
                        s[j].valid = false;
                        s[j].reason = loop
                        break
                    end
                    parentindex = s[parentindex].parent
                end
            end
#           check repetition within the same step
            if s[j].valid
                parentindex = j - 1;
                while s[parentindex].step == step
                    if comparestatuses(s[j],s[parentindex])
                        s[j].valid = false;
                        s[j].reason = repeat
                        break;
                    end
                    parentindex = parentindex - 1
                end
            end
        end
        if s[j].valid
            notanyvalid = false
            if comparestatuses(s[j],goalstatus)
                goalstatusreached = true
                s[j].reason = reachedgoal
            else
                s[j].reason = valid
            end
        end
    end
    return notanyvalid, goalstatusreached
end

function comparestatuses(x::Status,y::Status)::Bool
    equal::Bool = true
    for m in 1:length(x.passengers)
        equal = equal && x.passengers[m] == y.passengers[m]
    end
    return equal && x.shuttle == y.shuttle;
end;

function printstep(step::Int64,statuses::Array{Status},notanyvalid::Bool,goalstatusreached::Bool)
    reasons = ["REACHED GOAL","VALID","illegal","empty shuttle to empty place","loop","repeat occurrence within step"]
    passengernames = ["Rey","Kylo","Snoke"]
    
    if step == 0
        println("parent   Planet        shuttle     Ship")
        println("====== ==============  =======  =================")
    end
    println("       --------------  Step:$(step)   -----------------")
    for j in 1:length(statuses)
        if statuses[j].step == step
            parentindex = statuses[j].parent
            if parentindex > 0
            print(".",statuses[parentindex].substep)
        else
            print("  ")
        end
            print("->.",statuses[j].substep," ")
            for m in 1:length(statuses[j].passengers)
                if statuses[j].passengers[m]
                    print(" "^length(passengernames[m])," ")
                else
                    print(passengernames[m]," ")
                end
            end
            print("""    $(if statuses[j].shuttle "->" else "<-" end)     """)
            for m in 1:length(statuses[j].passengers)
                if !statuses[j].passengers[m]
                    print(" "^length(passengernames[m])," ")
                else
                    print(passengernames[m]," ")
                end
            end
            print("""     $(reasons[Int(statuses[j].reason)])\n""")
            if statuses[j].reason == reachedgoal
                goalstatusreached = true
            end
        end
    end
    if goalstatusreached
        println("                    ---- END ----")
    elseif notanyvalid
        println("                ---- NO SOLUTION ----")
    end
end;

In [2]:
# MAIN PROGRAM

s = Array{Status}(undef,100);
# initialize state space
for i in 1:length(s)
    s[i] = Status(-1,-1,-1,[false,false,false],false,false,initial)
end

nextindex = 1 #points to the last filled status in the array S
s[nextindex] = Status(0,1,0,[false,false,false],false,true,valid)
goalstatus   = Status(0,0,0,[true,true,true],true,true,reachedgoal)
goalstatusreached = false #end condition: goal status reached
notanyvalid       = false #end condition: last step did not generate any new valid status
nextstep          = 0 #generation step counter

# print initial status as Step 0.
printstep(nextstep,s,notanyvalid,goalstatusreached)
# either goal status reached 
# or the last step did not generate any new valid status
while !goalstatusreached && !notanyvalid
    nextstep = nextstep + 1
    nextindex = generate!(nextstep,nextindex,s);
    notanyvalid , goalstatusreached = test!(nextstep,s);
    printstep(nextstep, s, notanyvalid, goalstatusreached)
end

parent   Planet        shuttle     Ship
====== ==============  =======  =================
       --------------  Step:0   -----------------
  ->.1 Rey Kylo Snoke     <-                         VALID
       --------------  Step:1   -----------------
.1->.1     Kylo Snoke     ->     Rey                 VALID
.1->.2 Rey      Snoke     ->         Kylo            illegal
.1->.3 Rey Kylo           ->              Snoke      illegal
.1->.4 Rey Kylo Snoke     ->                         empty shuttle to empty place
       --------------  Step:2   -----------------
.1->.1 Rey Kylo Snoke     <-                         loop
.1->.2     Kylo Snoke     <-     Rey                 VALID
       --------------  Step:3   -----------------
.2->.1          Snoke     ->     Rey Kylo            VALID
.2->.2     Kylo           ->     Rey      Snoke      VALID
.2->.3     Kylo Snoke     ->     Rey                 loop
       --------------  Step:4   -----------------
.1->.1 Rey      Snoke     <-         Kylo    

TODO:
- replace Array(100) with dynamic size array - graph perhaps?